## Imports

In [25]:
# Connecting to the drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
## Importing all dependancies
import pandas as pd
import pickle
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
from torch.utils.data import DataLoader, Dataset
import tqdm
import torch.optim as optim

In [15]:
## Impoerting Node-Map Dict

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_node_map_dict','rb')as f:
  node_map_dict = pickle.load(f)

In [16]:
## Importing trained KG model to access embedddings

# KG Model Defination
class TransE(nn.Module):
    def __init__(self, num_entities, num_relations, embedding_dim):
        # The initialization acts like lookup layer where shallow random embeddings are generated for each node and relation
        super(TransE, self).__init__()
        self.entity_embeddings = torch.nn.Parameter(torch.randn(num_entities, embedding_dim))
        self.relation_embeddings = torch.nn.Parameter(torch.randn(num_relations, embedding_dim))

    def forward(self):
      # The function computes l2 norm of every row and devide the node entitity emdedding by their row-wise norms to normalize them
        self.entity_embeddings.data[:-1, :].div_(
            self.entity_embeddings.data[:-1, :].norm(p=2, dim=1, keepdim=True))
        return self.entity_embeddings, self.relation_embeddings

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_KG_model','rb')as f:
  trained_model = pickle.load(f)

## Definations

In [19]:
#Define the dataset and dataloader
class RatingDataset(Dataset):
    def __init__(self, user_ids, item_ids, ratings):
        self.user_ids = user_ids
        self.item_ids = item_ids
        self.ratings = ratings

    def __len__(self):
        return len(self.ratings)

    def __getitem__(self, idx):
        user_id = self.user_ids[idx]
        item_id = self.item_ids[idx]
        rating = self.ratings[idx]
        return {'user_id': user_id, 'item_id': item_id, 'rating': rating}

In [21]:
# Define the secondary model architecture
class CFNet(nn.Module):
    def __init__(self, num_users, num_items, embedding_dim):
        super(CFNet, self).__init__()
        self.user_embeddings = nn.Embedding(num_users, embedding_dim, _weight=user_embeddings)
        self.item_embeddings = nn.Embedding(num_items, embedding_dim, _weight=item_embeddings)
        self.fc1 = nn.Linear(embedding_dim * 2, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, user_ids, item_ids):
        user_embeds = self.user_embeddings(user_ids)
        item_embeds = self.item_embeddings(item_ids)
        x = torch.cat([user_embeds, item_embeds], dim=1)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

## Pipeline

### Creating User-POI rating train dictonary {user:[[poi,rating],[poi2,rating2]]}


In [3]:
# Accessing User Reviews
import pandas as pd
Visit_metadata = pd.read_csv(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/tucson_reviews.csv')
Visit_metadata

,review_id,user_id,business_id,stars,text
0,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3,Family diner. Had the buffet. Eclectic assortm...
1,UBp0zWyH60Hmw6Fsasei7w,4Uh27DgGzsp6PqrH913giQ,otQS34_MymijPTdNBoBdCw,4,The bun makes the Sonoran Dog. It's like a snu...
2,Jut1KvC3_mW6RGryns_wNQ,0pgYDWcqMiiHUln2JUux3A,6N9T23_iXkcOCh6boAXpIg,3,I will definitely go back and explore the menu...
3,onlgwy5qGDEzddsrnIvtWg,pYXeL0RCqus2IfhthYCOyA,W7NxQw8UYFR0HLPrI08tvw,4,Don't know what it is but If my tummy's feelin...
4,M7h8K7PrLrN8yQb3RhHLdA,CysTz612pTix75Fye6-D4Q,WLiqfxv_GhFFA5sm878a2w,5,Blew $80 on food for the family was worth it f...
...,...,...,...,...,...
154096,KKHdjeQkQbWHnUys7m6cLA,-lg4J2SOILBJOxqH4S0x4g,QpAVPiH1Yujksd_ReL-x-g,5,"Wow, this was one of the best meals I've ever ..."
154097,feNpfoo9wlxYVBG5d0y1mQ,dvu83QWKsnnbGvNVQ3mv7A,GxuxCctcz3Hyk0wnuly7vQ,4,This is now one of two Starbucks in the shoppi...
154098,dkGbETTcSQZTwHSnAMnLUw,_RmG_5kxRPgTWP7RptaFgQ,Bq0CQcwk5R8yhm-MGfHxCA,5,Bosnian food?? \n\n--- location. This is a HID...
154099,YGUyDcnAxZsUAjlK3QF99g,Qa5MBgb7Xyrr9Pg5bD4taQ,jh8j-DWqgWkbRe_a2XtKFQ,5,My secret to finding good food often starts by...


In [4]:
## Creating userwise visited POI rating dictonary

UP_dict = {}

for i in range(len(Visit_metadata)):
  user = Visit_metadata.iloc[i,1]
  if user not in UP_dict:
    UP_dict[user] = []
  poi = Visit_metadata.iloc[i,2]
  rating = Visit_metadata.iloc[i,3]
  UP_dict[user].append([poi,rating])

In [7]:
## Splitting User-POI visit dictonary into train and test
import tqdm

UP_dict_train = {}
UP_dict_test = {}

for user in tqdm.tqdm(UP_dict) :
  if user not in UP_dict_train:
    UP_dict_train[user] = []
    UP_dict_test[user] = []

  user_visits = len(UP_dict[user])
  test_count = int(user_visits*0.2)
  train_count = user_visits - test_count

  for record in UP_dict[user]:
    if train_count > 0 :
      UP_dict_train[user].append(record)
      train_count -= 1
    elif test_count > 0 :
      UP_dict_test[user].append(record)
      test_count -= 1

100%|██████████| 3365/3365 [00:00<00:00, 42114.25it/s]


In [14]:
import pickle
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/UP_dict_train','wb')as f:
  pickle.dump(UP_dict_train,f)

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/UP_dict_test','wb')as f:
  pickle.dump(UP_dict_test,f)

### Context Learning

In [17]:
# KG to secondary model indices mapping of users and POIs
user_embedding_list = []
node_index_user_index_mapper = {}
item_embedding_list = []
node_index_item_index_mapper = {}
user_pointer = 0
item_pointer = 0

user_ids = []
item_ids = []
ratings = []

for user in tqdm.tqdm(UP_dict_train):
  if node_map_dict[user] not in node_index_user_index_mapper:
    node_index_user_index_mapper[node_map_dict[user]]=user_pointer
    user_pointer +=1
    user_embedding_list.append(trained_model.entity_embeddings[node_map_dict[user]])

  for record in UP_dict_train[user]:
    if node_map_dict[record[0]] not in node_index_item_index_mapper:
      node_index_item_index_mapper[node_map_dict[record[0]]]= item_pointer
      item_pointer +=1
      item_embedding_list.append(trained_model.entity_embeddings[node_map_dict[record[0]]])

    user_ids.append(node_index_user_index_mapper[node_map_dict[user]])
    item_ids.append(node_index_item_index_mapper[node_map_dict[record[0]]])
    ratings.append(record[1])

# Saving the primary-secondary map
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/user_kg_index_secondary_id_map_tucson','wb') as f:
  pickle.dump(node_index_user_index_mapper,f)

with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/item_kg_index_secondary_id_map_tucson','wb') as f:
  pickle.dump(node_index_item_index_mapper,f)

100%|██████████| 3365/3365 [00:00<00:00, 6183.34it/s]


In [18]:
# Constructing User-POI tensors
user_embeddings = torch.stack(user_embedding_list)
item_embeddings = torch.stack(item_embedding_list)

user_ids = torch.tensor(user_ids)
item_ids = torch.tensor(item_ids)
ratings = torch.tensor(ratings)

In [20]:
dataset = RatingDataset(user_ids, item_ids, ratings)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

In [22]:
#Define loss function and optimizer
criterion = nn.MSELoss()
model = CFNet(len(user_embedding_list),len(item_embedding_list),50)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [23]:
# Train the model
for epoch in range(100):
    for bno,batch in enumerate(tqdm.tqdm(dataloader)):
        user_ids = batch['user_id']
        item_ids = batch['item_id']
        ratings = batch['rating']

        optimizer.zero_grad()
        outputs = model(user_ids, item_ids)
        loss = criterion(outputs.squeeze(), ratings.float())
        loss.backward()
        optimizer.step()
        if bno%50 == 0:
          print("Loss: ",loss)

  1%|          | 10/974 [00:00<00:22, 42.40it/s]

Loss:  tensor(16.2147, grad_fn=<MseLossBackward0>)


  7%|▋         | 70/974 [00:00<00:10, 90.00it/s]

Loss:  tensor(7.0306, grad_fn=<MseLossBackward0>)


 11%|█▏        | 111/974 [00:01<00:08, 96.06it/s]

...

 94%|█████████▎| 911/974 [00:11<00:00, 72.15it/s]

Loss:  tensor(0.1715, grad_fn=<MseLossBackward0>)


 99%|█████████▊| 960/974 [00:12<00:00, 74.30it/s]

Loss:  tensor(0.1925, grad_fn=<MseLossBackward0>)


100%|██████████| 974/974 [00:12<00:00, 77.42it/s]


In [24]:
## Saving secondary model trained embeddings
with open(r'/content/drive/My Drive/GRS_For_POI_Experiments/Post_WITS/Tucson City Data/secondary_model_tucson','wb') as f:
  pickle.dump(model,f)

## Inference